In [1]:
from PyQt5 import QtWidgets, QtGui, QtCore
from HW2_UI import Ui_MainWindow
import sys
import numpy as np
import cv2
import math
import glob

In [2]:
def Find_Draw():
    coin01 = cv2.imread('Q1_Image/coin01.jpg')
    coin01_gray = cv2.cvtColor(coin01, cv2.COLOR_RGB2GRAY)

    kernel_size = 11
    blur_coin01_gray = cv2.GaussianBlur(coin01_gray,(kernel_size, kernel_size), 0)

    low_threshold = 30
    high_threshold = 150
    coin01_edge = cv2.Canny(blur_coin01_gray, low_threshold, high_threshold)

    binary, coin01_contours, hierarchy = cv2.findContours(coin01_edge, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(coin01, coin01_contours, -1, (0,0,255), thickness = 2)

    coin02 = cv2.imread('Q1_Image/coin02.jpg')
    coin02_gray = cv2.cvtColor(coin02, cv2.COLOR_RGB2GRAY)

    kernel_size = 11
    blur_coin02_gray = cv2.GaussianBlur(coin02_gray,(kernel_size, kernel_size), 0)

    low_threshold = 30
    high_threshold = 150
    coin02_edge = cv2.Canny(blur_coin02_gray, low_threshold, high_threshold)

    binary, coin02_contours, hierarchy = cv2.findContours(coin02_edge, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(coin02, coin02_contours, -1, (0,0,255), thickness = 2)

    cv2.imshow("coin01_contours", coin01)
    cv2.imshow("coin02_contours", coin02)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()
def Find_corners():
    w = 11
    h = 8
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    objp = np.zeros((w*h,3), np.float32)
    objp[:,:2] = np.mgrid[0:w,0:h].T.reshape(-1,2)

    objpoints = [] 
    imgpoints = [] 
    images = glob.glob('Q2_Image\*.bmp')

    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (w,h),None)
        if ret == True:  
            cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
            objpoints.append(objp)
            imgpoints.append(corners)
            cv2.drawChessboardCorners(img, (w,h), corners, ret)
            img = cv2.resize(img, (960, 960))
            cv2.namedWindow("findCorners",cv2.WINDOW_AUTOSIZE)
            cv2.imshow('findCorners',img)
            cv2.waitKey(0)
    cv2.destroyAllWindows()
def Find_Intrinsic():
    w = 11
    h = 8
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    objp = np.zeros((w*h,3), np.float32)
    objp[:,:2] = np.mgrid[0:w,0:h].T.reshape(-1,2)

    objpoints = [] 
    imgpoints = [] 
    images = glob.glob('Q2_Image\*.bmp')

    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (w,h),None)
        if ret == True:  
            cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
            objpoints.append(objp)
            imgpoints.append(corners)
             
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    print("Intrinsic Matrix:")
    print(mtx)
def Find_Distortion():
    w = 11
    h = 8
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    objp = np.zeros((w*h,3), np.float32)
    objp[:,:2] = np.mgrid[0:w,0:h].T.reshape(-1,2)

    objpoints = [] 
    imgpoints = [] 
    images = glob.glob('Q2_Image\*.bmp')

    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (w,h),None)
        if ret == True:  
            cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
            objpoints.append(objp)
            imgpoints.append(corners)
            cv2.drawChessboardCorners(img, (w,h), corners, ret) 
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    print("Distortion Matrix:")
    print(dist)
def eulerAnglesToRotationMatrix(theta, tran) :
    R_x = np.array([[1, 0, 0],
                    [0, math.cos(theta[0]), -math.sin(theta[0])],
                    [0, math.sin(theta[0]), math.cos(theta[0])]])
    
    R_y = np.array([[math.cos(theta[1]), 0, math.sin(theta[1])],
                    [0, 1, 0],
                    [-math.sin(theta[1]), 0, math.cos(theta[1])]])
    
    R_z = np.array([[math.cos(theta[2]), -math.sin(theta[2]), 0],
                    [math.sin(theta[2]), math.cos(theta[2]), 0],
                    [0, 0, 1]])
    
    R = np.dot(R_z, np.dot( R_y, R_x ))
    extrinsic_matrix = np.concatenate((R,tran), axis=1)
    return extrinsic_matrix
def Find_Extrinsic(number):
    w = 11
    h = 8
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    objp = np.zeros((w*h,3), np.float32)
    objp[:,:2] = np.mgrid[0:w,0:h].T.reshape(-1,2)

    objpoints = [] 
    imgpoints = [] 
    images = glob.glob('Q2_Image\*.bmp')

    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (w,h),None)
        if ret == True:  
            cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
            objpoints.append(objp)
            imgpoints.append(corners)
            cv2.drawChessboardCorners(img, (w,h), corners, ret) 
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    print("Extrinsic Matrix:")
    print(eulerAnglesToRotationMatrix(rvecs[number], tvecs[number]))
# picture 2 unstable
def draw(img, pyramidimgpoints):
    for i in range(4):
        for j in range(i + 1,4):
            cv2.line(img, (int(pyramidimgpoints[i,:,:][0][0]),int(pyramidimgpoints[i,:,:][0][1])),
                     (int(pyramidimgpoints[j,:,:][0][0]),int(pyramidimgpoints[j,:,:][0][1]))
                     , (0, 0, 255), 5)
    img = cv2.resize(img, (960, 960))
    cv2.imshow('My Image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
def Augmented_Reality():
    w = 11
    h = 8
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    objp = np.zeros((w*h,3), np.float32)
    objp[:,:2] = np.mgrid[0:w,0:h].T.reshape(-1,2)

    objpoints = [] 
    imgpoints = [] 
    pyramidimgpoints = []
    images = glob.glob(r'Q3_Image\*.bmp')

    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (w,h),None)
        if ret == True:
            cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
            objpoints.append(objp)
            imgpoints.append(corners)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    i = 0
    pyramidpoints = np.array([[1.,1.,0.],[5.,1.,0.],[3.,3.,-3.],[3.,5.,0.]])
    for fname in images:
        img = cv2.imread(fname)
        pyramidimgpoints, jacobian = cv2.projectPoints(pyramidpoints, rvecs[i], tvecs[i], mtx, dist)
        draw(img, pyramidimgpoints)
        i += 1
class Map:
    def __init__(self, disparity):
        self.disparity = disparity
        self.Map = np.array([])
        self.showMap = np.array([])
    def depth(self, disparity):
        baseline = 178
        focal_length = 2826
        return focal_length*baseline/(disparity)
    
    def norm(self):
        amin, amax = self.disparity.min(), self.disparity.max()
        self.Map = (self.disparity - amin)/(amax - amin)*255
    
    def OnMouseAction(self, event,x,y,flags,param):
        if event == cv2.EVENT_LBUTTONDOWN:
            x_index = x*2
            y_index = y*2
            dis = 'Disparity: '+ str(self.disparity[y_index][x_index]+123)+' Pixel'
            depth = 'Depth: '+ str(int(self.depth(self.disparity[y_index][x_index]+123)))+' mm'
            cv2.rectangle(self.showMap, (1109 ,860), (1406 ,953), (0, 0, 0), -1)
            cv2.putText(self.showMap, dis, (1130 ,900), cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 0.8, (255, 255, 255), 1,cv2.LINE_AA)
            cv2.putText(self.showMap, depth, (1130 ,930), cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 0.8, (255, 255, 255), 1,cv2.LINE_AA)
            
    def show(self):
        self.showMap = cv2.resize(self.Map, (1410, 960))
        cv2.namedWindow('Stereo Disparity Map')
        cv2.setMouseCallback('Stereo Disparity Map', self.OnMouseAction)
        while(1):
            cv2.imshow('Stereo Disparity Map', self.showMap)
            k = cv2.waitKey(1)
            if k == ord('q'):
                break
        cv2.destroyAllWindows()
        
def Stereo():
    imgL = cv2.imread('Q4_Image\imgL.png',cv2.IMREAD_GRAYSCALE)
    imgR = cv2.imread('Q4_Image\imgR.png',cv2.IMREAD_GRAYSCALE)
    stereo = cv2.StereoBM_create(numDisparities=272, blockSize=21)
    disparity = stereo.compute(imgL,imgR)
    output = Map(disparity)
    output.norm()
    output.show()

In [3]:
class MainWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super(MainWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)
        self.ui.pushButton.clicked.connect(Find_Draw)
        self.ui.pushButton_2.clicked.connect(self.Count_coin)
        self.ui.pushButton_3.clicked.connect(Find_corners)
        self.ui.pushButton_4.clicked.connect(Find_Intrinsic)
        self.ui.pushButton_5.clicked.connect(Find_Distortion)
        self.ui.pushButton_8.clicked.connect(self.Extrinsic)
        self.ui.pushButton_6.clicked.connect(Augmented_Reality)
        self.ui.pushButton_7.clicked.connect(Stereo)     
        
    def Extrinsic(self):
        number = self.ui.comboBox.currentIndex()
        Find_Extrinsic(number)
        
    def Count_coin(self):
        self.ui.label.setText("There are 5 coins in coin01.jpg")
        self.ui.label_2.setText("There are 20 coins in coin02.jpg")

In [ ]:
if __name__ == '__main__':
    app = QtWidgets.QApplication([])
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())

Intrinsic Matrix:
[[2.22339353e+03 0.00000000e+00 1.02888550e+03]
 [0.00000000e+00 2.22295242e+03 1.04050529e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Intrinsic Matrix:
[[2.22339353e+03 0.00000000e+00 1.02888550e+03]
 [0.00000000e+00 2.22295242e+03 1.04050529e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Matrix:
[[-0.12851724  0.10091855 -0.00062626  0.00043284 -0.02310165]]
Extrinsic Matrix:
[[-0.93604713 -0.20114182 -0.28871738  6.78773089]
 [ 0.08119393 -0.9218357   0.37898086  3.36118118]
 [-0.34237889  0.33130185  0.87921316 16.70795214]]
